In [1]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



#x: array([ 12.        , 116.21503406,   1.1       ,   2.2       ,3.50562199])

def optim(co2_t, co2_k, co2_cp, co2_rho, co2_tempInit):

#PCM_k, PCM_c, PCM_h, co2_h, coeff
    def diffusivity(k,rho,cp):
        return k/(rho*cp)

    # Given parameters (_t thickness m) (_k conductivity W/mK) (_c sp. heat capacity at const pressure KJ/kg) (_lh latent heat KJ/kg) (_rho density kg/m^3)

    # Outer Insulation
    minWool_t = 50e-3
    minWool_k = 34e-3
    minWool_rho = 160.
    minWool_cp = 1030.
    minWool_tempInit = 18

    # Dry Ice
    # co2_t = 60e-3
    # co2_k = 16e-3
    # co2_cp = 0.658e3
    # co2_rho = 1.795
    # # co2_h = 0.1
    # co2_tempInit = -10.

    # PCM

    PCM_k = 20.
    PCM_c = 115.95440697
    # PCM_h = 3
    PCM_t = 30e-3
    # PCM_cp_liq = 3.40
    # PCM_cp_sol = 1.87
    # PCM_k_liq = 0.53
    # PCM_k_sol = 5.26
    # PCM_lh = 327.
    PCM_rho = 1043.
    PCM_tempInit = -25.

    # Outside/ Ambient parameters

    amb_h = 5. #w/m2K
    amb_temp = 25. #c
    coeff = 3.45735793


    minWool_Diff = diffusivity(minWool_k,minWool_rho,minWool_cp)
    co2_diff = diffusivity(co2_k,co2_rho,co2_cp)
    # PP_diff = diffusivity(PP_k, PP_rho, PP_c)
    PCM_diff = diffusivity(PCM_k, PCM_rho, PCM_c)


    biot_minWool = amb_h*minWool_t/minWool_k
    biot_co2 = 2.2*co2_t/co2_k
    biot_PCM = 1.1*PCM_t/12.

    def eigenRoots(Bi):

        val = []
        CN = []
        
        for n in range (1,500): val.append(fsolve(lambda y: y*np.tan(y) - Bi, n))
        uniqueRoots = np.unique(np.around(np.concatenate(val,axis=0), decimals=4)) #this nested function initially flattens the list of ndarray objects into numpy float64 elements which are then rounded to 4 decimal places and then the unique values of them are extracted using np.unique() function 
        for n in range(0,100):            
            CN.append((4*np.sin(uniqueRoots[n])) / ((2*uniqueRoots[n]) + np.sin(2*uniqueRoots[n])))           
        return uniqueRoots,CN

    def tAtXandT(cn, z, a, t, l, xS):   
        fo =  a * t / l
        return cn * np.exp(-z**2 * fo) * np.cos(z*xS)

    minWoolTvst = []
    co2Tvst = []
    PCMTvst = []
    x = 0.

    U1,R1 = eigenRoots(biot_minWool)
    U2,R2 = eigenRoots(biot_co2)
    U3,R3 = eigenRoots(biot_PCM)

    t1 = 1
    t = t1*3600
    theta = 0.0
    for i in range (0,100):
        theta += tAtXandT(R1[i], U1[i], minWool_Diff, t, minWool_t, x)
        minWool_tempxt = (minWool_tempInit - amb_temp)*theta + amb_temp
    
    
    theta = 0.0 
    for i in range (0,100):
        theta += tAtXandT(R2[i], U2[i], co2_diff, t, co2_t, x)
        co2_tempxt = (co2_tempInit - 20)*theta + 20
    
    
    theta = 0.0 
    for i in range (0,100):
        theta += tAtXandT(R3[i], U3[i], PCM_diff, t, PCM_t, x)
        PCM_tempxt = (PCM_tempInit - co2_tempxt)*theta + co2_tempxt + coeff
    
    return co2_tempxt

In [2]:
probDist = np.random.default_rng()
nRuns = 1000
#co2_t, co2_k, co2_cp, co2_rho, co2_tempInit

t_corr = probDist.uniform(15e-3,30e-3,nRuns).round(4)
k_corr = probDist.uniform(5e-3,16e-3,nRuns).round(4)
cp_corr = probDist.uniform(0.658e3,2.3e3,nRuns).round(4)
rho_corr = probDist.uniform(1.977,6.0,nRuns).round(4)
# tempInit_corr = probDist.uniform(-1,1,nRuns).round(4)
PCMt_final_corr = np.zeros_like(k_corr)
PCMt_diff_corr = np.zeros_like(k_corr)

for i in range (0,nRuns):
    PCMt_final_corr[i] = optim(t_corr[i],k_corr[i],cp_corr[i],rho_corr[i],1).round(4)

In [3]:
# nRuns = 10
# #co2_t, co2_k, co2_cp, co2_rho, co2_tempInit

# t_corr = np.zeros([nRuns,1])
# k_corr = np.zeros([nRuns,1])
# cp_corr = np.zeros([nRuns,1])
# rho_corr = np.zeros([nRuns,1])
# tempInit_corr = np.zeros([nRuns,1])
# PCMt_corr = np.zeros([nRuns,1])

# for i in range (0,nRuns):
#     t_corr[i] = probDist.triangular(25e-3,30e-3,50e-3)
#     k_corr[i] = probDist.triangular(14e-3,16e-3,18e-3)
#     cp_corr[i] = probDist.triangular(0.99e3,1.006e3,5.1e3)
#     rho_corr[i] = probDist.triangular(1.1,1.276,5.4)
#     tempInit_corr[i] = probDist.triangular(-3.,2,5)
    
#     PCMt_corr[i] = optim(t_corr[i],k_corr[i],cp_corr[i],rho_corr[i],tempInit_corr[i])





In [4]:
# nRuns = 1000
# #co2_t, co2_k, co2_cp, co2_rho, co2_tempInit

# t_corr = probDist.uniform(28e-3,30e-3,nRuns).round(4)
# k_corr = probDist.uniform(5e-3,16e-3,nRuns).round(4)
# cp_corr = probDist.uniform(0.65e3,2.5e3,nRuns).round(4)
# rho_corr = probDist.uniform(1e3,1.1e3,nRuns).round(4)
# tempInit_corr = probDist.uniform(-1,1,nRuns).round(4)
# PCMt_final_corr = np.zeros_like(k_corr)
# PCMt_diff_corr = np.zeros_like(k_corr)

# for i in range (0,nRuns):
#     PCMt_final_corr[i] = optim(30e-3,k_corr[i],1.5e3,5,1.5).round(4)
#     PCMt_diff_corr[i] = (( PCMt_final_corr[i] - 1.5) / 1.5) * 100

In [6]:
corr_table = pd.DataFrame({ 'Final temp': PCMt_final_corr.ravel(), 'Thickness': t_corr.ravel(), 'k': k_corr.ravel(), 'cp': cp_corr.ravel(), 'rho': rho_corr.ravel()})
# corr_table['PCM Temp'] = corr_table['PCM Temp'].astype('float')
# corr_table['T'] = corr_table['T'].astype('float')
# corr_table['k'] = corr_table['k'].astype('float')
# corr_table['cp'] = corr_table['cp'].astype('float')
# corr_table['rho'] = corr_table['rho'].astype('float')
# corr_table['PCM Temp', 'T','k','cp','rho'] = corr_table['PCM Temp', 'T','k','cp','rho'].astype('float')
corr_matrix = corr_table.corr('pearson').round(3)
corr_matrix.style.background_gradient(cmap='coolwarm')

# sns.heatmap(corr_matrix)

# sns.heatmap(corr_matrix, 
#             xticklabels=corr_matrix.columns.values,
#             yticklabels=corr_matrix.columns.values)

,Final temp,Thickness,k,cp,rho
Final temp,1.000000,-0.242000,0.423000,-0.646000,-0.507000
Thickness,-0.242000,1.000000,-0.030000,0.036000,-0.066000
k,0.423000,-0.030000,1.000000,-0.005000,0.007000
cp,-0.646000,0.036000,-0.005000,1.000000,-0.029000
rho,-0.507000,-0.066000,0.007000,-0.029000,1.000000
